In [1]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00


In [2]:
# pip install transformers accelerate

from transformers import AutoTokenizer
import transformers
import torch

model = "varshil27/Llama-2-7b-finetune-2"
prompt = "These are patient's Symptoms Cough, Throat pain , Headache and patient is diagnosed with Bronchitis. What are the top 5 most likely prescribed medicines corresponding to these symptoms and disease? Output only medicine names."

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    f'<s>[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:142

Result: <s>[INST] These are patient's Symptoms Cough, Throat pain , Headache and patient is diagnosed with Bronchitis. What are the top 5 most likely prescribed medicines corresponding to these symptoms and disease? Output only medicine names. [/INST] Based on the symptoms and disease mentioned by the patient, the most likely medicine prescriptions are as follows:

1. Xoltar LS Syrup: This is an antioxidant and is used in chronic bronchitis and lung infections. It reduces the swelling in the airways and thus, helps in normal breathing. It relaxes the airways and thus, widens them, making it easier for air to pass through. This makes breathing easier and reduces the frequency of strokes or spasms in the muscles of the respiratory tract. This medicine is usually given 2-3 times a day. Take


In [3]:
from collections import defaultdict
from flask import Flask, request, jsonify, Response
import pandas as pd
import numpy as np
import pickle
import json
import os

In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install pyngrok
!ngrok authtoken 2XIv3b9zcdpvNSIjBQreicsv4PB_839F2HmjE83JJYHiamr6F
from pyngrok import ngrok
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=38a6c4bdfd3680525a3f610630b84de1e7a6979f72b41043145812529472a5e8
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
app = Flask(__name__)
run_with_ngrok(app)

In [6]:
@app.route('/predict_medicine', methods=['POST'])
def predictor():
  data = request.get_json()
  Symptoms = data["Symptoms"]
  Disease = data["Disease"]
  prompt = f"These are patient's Symptoms {Symptoms} and disease patient is diagnosed with {Disease}. What are the top 5 most likely medicines prescribed to patients for this condition? Output only medicine names."

  sequences = pipeline(
    f'<s>[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
  )
  Medicines = ""
  for seq in sequences:
      Medicines += (f"{seq['generated_text']}")
  Medicines = Medicines.split("[/INST] ")

  Medicines = (Medicines[1])

  Medicines = Medicines[:-1]
  Medicines = Medicines.split(', ')
  print(Medicines)

  return jsonify({"Medicines": Medicines})




In [7]:
if __name__ == "__main__":
    try:
        app.run()
    except Exception as e:
        print(f"Error: {e}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3d4a-104-197-99-223.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
